In [4]:
#Paquetes necesarios:
import pandas as pd
import time
from bs4 import BeautifulSoup as bs
from selenium import webdriver
import json

In [7]:
PATH = "./chromedriver.exe"
driver = webdriver.Chrome(PATH) #Escribe el PATH de geckodriver
driver.get("https://serviciosweb.itam.mx/EDSUP/BWZKSENP.P_Horarios1?s=1809") #Cambia cada periodo
soup=bs(driver.page_source, "html.parser")

numCourses = len(soup("option"))

dataFrame = pd.DataFrame()

courses = {}

# Modificado para obtener todas las materias
#for k in range(0, 2):#numCourses):
for k in range(400, numCourses):

    boton=driver.find_element_by_xpath("/html/body/div[3]/form/input[2]")
    materiak=driver.find_element_by_xpath('/html/body/div[3]/form/select/option[{}]'.format(k+1))
    materiak.click()
    
    boton.click()
    
    tablak=pd.read_html(driver.page_source)
    data = tablak[2][:][1:]

    for i in range(len(data)):
        row = data[:][i:i+1];

        # Get the keys for the dictionary
        # Key = Department + code + name
        key = row[0].values[0] + '-' + row[1].values[0] + ' ' + row[4].values[0]
        group = int(row[2].values[0])

        #Built the object
        newClass = {
            "type": row[3].values[0],
            "teacher": row[5].values[0], 
            "credits": row[6].values[0],
            "time": row[7].values[0].split("-"),
            "days": row[8].values[0].split(" "),
            "room": row[9].values[0],
            "campus": row[10].values[0],
            "comments": row[11].values[0]
        }
                    
        #Validate that keys exist, if not we add it to the dict
        if key not in courses:
            courses[key] = {group : [newClass]}
        elif group not in courses[key]:
            courses[key][group] = [newClass]
        else:
            courses[key][group].append(newClass)
        
    driver.back()

driver.quit()

with open('courses.json', 'w') as fp:
    json.dump(courses, fp)
    #json.dump(courses, fp, indent=4)